In [30]:
import astropy
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from matplotlib.pyplot import figure

In [46]:
#APOKASC file & file variables
hdul = fits.open(r"/Users/amber3/Desktop/SSI/APOKASC_cat_v6.7.2.fits")
#hdul.info()
hdu = hdul[1]
data = hdu.data
#print(np.shape(data))
KIC_A = hdu.data['KEPLER_INT'] 
Logg_A = hdu.data['DR16_LOGG_COR']
Logg_err = hdu.data['DR16_LOGG_COR_ERR']
Teff_A = hdu.data['DR16_TEFF_COR']
Teff_err = hdu.data['DR16_TEFF_COR_ERR']
FeH_A = hdu.data['DR16_M_H_COR']
#print(Logg_A)
FeH_err = hdu.data['DR16_M_H_COR_ERR']
R_APO3 = hdu.data['APOKASC3P_RADIUS']
R_APO3_err = hdu.data['APOKASC3P_RADIUS_ERR']
R_Gaia = hdu.data['GAIA_RADIUS'] #no error on this radius
M_APO3 = hdu.data['APOKASC3P_MASS']
M_APO3_err = hdu.data['APOKASC3P_MASS_ERR']

In [47]:
#APOLLINAIRE file & file variables (b = background)
hdul_b = fits.open(r"/Users/amber3/Desktop/SSI/Selected_stars_final.fits")
hdu_b = hdul_b[0]
data_b = hdu_b.data
KIC_b = np.array(data_b[:,0,0])
print(KIC_b)

[ 1027337.  1161447.  1163453. ... 12784683. 12784948. 12884116.]


In [48]:
common_KIC, A_indices, b_indices = np.intersect1d(KIC_A, KIC_b, return_indices=True)
#print(np.shape(A_indices), np.shape(b_indices))
good_indices = np.where(Teff_A[A_indices]*R_APO3[A_indices] > 0)
b_indices = b_indices[good_indices]
A_indices = A_indices[good_indices]
KIC_b = KIC_b[b_indices]
Amp_HV1 = np.array(data_b[b_indices,0,6])
Amp_HV1_err = np.array(data_b[b_indices,1,6])
Amp_HV2 = np.array(data_b[b_indices,0,9])
Amp_HV2_err = np.array(data_b[b_indices,1,9])
Freq_HV1 = np.array(data_b[b_indices,0,7])
Freq_HV1_err = np.array(data_b[b_indices,1,7])
Freq_HV2 = np.array(data_b[b_indices,0,10])
Freq_HV2_err = np.array(data_b[b_indices,1,10])
Evol_stage = np.array(data_b[b_indices,0,3])
#print(KIC_A, KIC_b)

(4000,) (4000,)
[10000151 10000162 10000198 ...  9996342  9996347  9996595] [ 1027337.  1161447.  1163453. ... 12784683. 12784948. 12884116.]
